In [10]:
import nltk
import pandas as pd
from youtube_transcript_api import YouTubeTranscriptApi


In [11]:
df1 = pd.read_csv('Submissions.csv')

In [13]:
len(df1)

6616

In [14]:
df1 = df1[df1.Valid == 1]

In [15]:
len(df1)

5618

In [16]:
df1['YouTube link '].head(15)

5                    https://youtu.be/eyr8ucTK2Zw
7                    https://youtu.be/e1EGkUii-GA
8                    https://youtu.be/U9ftxDY1nhY
10                   https://youtu.be/9wNVwgqf-eI
11                    http://youtu.be/WymADeTTP4w
12                   https://youtu.be/jLdhDpC1L9k
14                   https://youtu.be/AMPMj6XcENE
16                   https://youtu.be/M6gGbrCmR_4
17                   https://youtu.be/P4iWrNjRKTM
19                   https://youtu.be/NcjpVTIto3E
20                   https://youtu.be/fPqIzmiUUYg
21    https://www.youtube.com/watch?v=NBMEgeu9HDY
22                   https://youtu.be/ChyYDvBb124
23                   https://youtu.be/E3Bnrlfjvx4
24    https://www.youtube.com/watch?v=UkzxuACvp0s
Name: YouTube link , dtype: object

In [17]:
df = df1.dropna(subset =['YouTube link '])

In [18]:
len(df)

5585

In [19]:
# df.dropna(how='all', axis=1)

In [20]:
df.dtypes

Semester           object
Course             object
U-Id               object
name               object
 email address     object
YouTube link       object
Link exist        float64
Valid             float64
Duration          float64
Dashboard         float64
Comments           object
Reviewd by         object
Task               object
dtype: object

In [21]:
df['YouTube link '] = df['YouTube link '].astype(str)

<ipython-input-21-79a7878b3124>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['YouTube link '] = df['YouTube link '].astype(str)


In [22]:
df.dtypes

Semester           object
Course             object
U-Id               object
name               object
 email address     object
YouTube link       object
Link exist        float64
Valid             float64
Duration          float64
Dashboard         float64
Comments           object
Reviewd by         object
Task               object
dtype: object

### The below functions parses the link the looks at all the places the video_id might be embedded in the list and extracts it.

In [23]:
from urllib.parse import urlparse, parse_qs

def extract_video_id(url):
    query = urlparse(url)
    if query.hostname == 'youtu.be': return query.path[1:]
    if query.hostname in ('www.youtube.com', 'youtube.com'):
        if query.path == '/watch': return parse_qs(query.query)['v'][0]
        if query.path[:7] == '/embed/': return query.path.split('/')[2]
        if query.path[:3] == '/v/': return query.path.split('/')[2]
    return "video not on Youtube or Does not exist"

### Extract the video ids for all the links in dataset. IF link does not exist feed 'video not on Youtube or Does not exist' in its place

In [24]:
# for i in df1.index:
   # link = df.loc('YouTube link ',i)
df['video_id'] = df['YouTube link '].apply(extract_video_id)

<ipython-input-24-84b079665548>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['video_id'] = df['YouTube link '].apply(extract_video_id)


In [25]:
df['video_id'].count()

5585

In [26]:
pd.set_option('display.max_rows', None)

df['video_id']

5                                        eyr8ucTK2Zw
7                                        e1EGkUii-GA
8                                        U9ftxDY1nhY
10                                       9wNVwgqf-eI
11                                       WymADeTTP4w
12                                       jLdhDpC1L9k
14                                       AMPMj6XcENE
16                                       M6gGbrCmR_4
17                                       P4iWrNjRKTM
19                                       NcjpVTIto3E
20                                       fPqIzmiUUYg
21                                       NBMEgeu9HDY
22                                       ChyYDvBb124
23                                       E3Bnrlfjvx4
24                                       UkzxuACvp0s
26                                       t0VUL7FWB-k
27                                       bagO_TR5CYw
28                                       aExRpXPFoPA
30                                       aUxCA

### Get Transcript of video from youtube , segregate the words , remove unwanted stopwords using NLTK, return a string of words clean of all stopwords

In [28]:
def transcript(video_id):
    try :
        print(video_id)
        m = YouTubeTranscriptApi.get_transcript(video_id)
        line = []
        new_line=[]
        stopwords_line=""
        for word in m:
#             line.append(word['text'])
            for w in word['text'].split(" "):
                line.append(w)
#             line = line + (word['text'])
        stopwords_line=remove_stopwords(line)
        return stopwords_line
    except Exception as e :
        print("----------------------------------------------")
        print(video_id)
        print(e)
        print("----------------------------------------------")
        return 'Nosubtitles'

In [29]:
from nltk.corpus import stopwords

def remove_stopwords(word_list):
        processed_word_list = []
        line = ""
        for word in word_list:
            word = word.lower() # in case they arenet all lower cased
            if word not in stopwords.words("english") and len(word) >=3 :
                processed_word_list.append(word)
        for word in processed_word_list:
            line = line + word + " "
        return line

In [ ]:
for i in df.index:
    df['transcript'] = df['video_id'].apply(transcript)

eyr8ucTK2Zw
e1EGkUii-GA
U9ftxDY1nhY
9wNVwgqf-eI
WymADeTTP4w
jLdhDpC1L9k
AMPMj6XcENE
----------------------------------------------
AMPMj6XcENE

Could not retrieve a transcript for the video https://www.youtube.com/watch?v=AMPMj6XcENE! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue at https://github.com/jdepoix/youtube-transcript-api/issues. Please add which version of youtube_transcript_api you are using and provide the information needed to replicate the error. Also make sure that there are no open issues which already describe your problem!
----------------------------------------------
M6gGbrCmR_4


In [ ]:
0ui7c726Kqo

In [ ]:
df['transcript']

In [ ]:
df

In [ ]:
df.to_csv(r'MLDataset.csv', index = False)

In [20]:
m=df['video_id'].tolist()

In [ ]:
df['transcript']=YouTubeTranscriptApi.get_transcripts(m, languages=['en'], continue_after_error=True, proxies=None, cookies=None)

In [ ]:
for transcript in df['transcript']:
    print(transcript)